<a href="https://www.kaggle.com/code/oswind/stockchat-agents-edition?scriptVersionId=282727108" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [1]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb.
os.environ["ANONYMIZED_TELEMETRY"] = "false"
# Check for kaggle environment.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-ai-generativelanguage pydrive2 tensorflow tensorflow-decision-forests cryptography pyOpenSSL langchain langchain-core nltk ray click google-generativeai google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway
    !pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all] google-adk google-adk[eval] google-cloud-translate
    from kaggle_secrets import UserSecretsClient # type: ignore
    from jupyter_server.serverapp import list_running_servers # type: ignore
else:
    # Mock the kaggle secrets client.
    class UserSecretsClient:
        @classmethod
        def set_secret(cls, id: str, value: str):
            os.environ[id] = value
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core "lmnr[all]" browser-use ollama google-adk "google-adk[eval]"
    from browser_use import Agent as BrowserAgent

import ast, chromadb, json, logging, pandas, platform, pytz, re, requests, sys, threading, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google.adk.apps.app import App
from google.adk.sessions import InMemorySessionService, BaseSessionService as SessionService, Session
from google.adk.runners import Runner, Event
from google import genai
from google.api_core import retry, exceptions
from google.genai.models import Models
from google.genai import types, errors
from IPython.display import Markdown, display, HTML
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType, NamedTuple
from wikipedia.exceptions import DisambiguationError, PageError

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 73.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper for generation not run through the below api-helper.
is_retriable = lambda e: (isinstance(e, errors.APIError) and e.code in {429, 503, 500})
Models.generate_content = retry.Retry(predicate=is_retriable)(Models.generate_content)
Models.embed_content = retry.Retry(predicate=is_retriable)(Models.embed_content)

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A python api-helper with model fail-over/chaining/retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Api:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-preview-09-2025": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.0-flash-exp": GeminiModel([10,10,10,10],[.25,.25,.25,.25],[200,500,500,500]), # latest w/thinking: 10 RPM/250K TPM/200 RPD
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable wo/thinking: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-flash-lite-preview-09-2025": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.125,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
    }
    gen_local = ["gemma3n:e4b","gemma3:12b-it-qat"]
    default_local = 0
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    embed_local = False
    error_total = 0
    min_rpm = 3
    min_tpm = 40000
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 45.0
    dt_rpm = 60.0

    @classmethod
    def get(cls, url: str):
        # Create a header matching the OS' tcp-stack fingerprint.
        system_ua = None
        match platform.system():
            case 'Linux':
                system_ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
            case 'Darwin':
                system_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 15_7_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0 Safari/605.1.15'
            case 'Windows':
                system_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
        try:
            request = requests.get(url, headers={'User-Agent': system_ua})
            if request.status_code != requests.codes.ok:
                print(f"Api.get() returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value
    
    class Env(NamedTuple): # Make init args immutable.
        CLIENT: genai.Client
        API_LIMIT: int
        GEN_DEFAULT: str

    def __init__(self, with_limit: Limit | int, default_model: str):
        if default_model in self.gen_model.keys():
            self.write_lock = threading.RLock()
            try:
                if isinstance(with_limit, int) and with_limit in [id.value for id in Api.Limit]:
                    limit = with_limit
                else:
                    limit = with_limit.value
            except Exception as e:
                print(f"Api.__init__: {with_limit} is not a valid limit")
            else:
                self.args = Api.Env(
                    genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY")),
                    limit, default_model)
            self.m_id = list(self.gen_model.keys()).index(default_model)
            self.default_model.append(default_model)
            self.update_quota()
            self.s_embed = GeminiEmbedFunction(self.args.CLIENT, semantic_mode = True) # type: ignore
            logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate
        else:
            print(f"Api.__init__: {default_model} not found in gen_model.keys()")
        

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0] if not self.embed_local else "embeddinggemma:latest"

    def push_default_model(self, model_id: str):
        if model_id in self.gen_model.keys():
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.append(model_id)
            self.m_id = list(self.gen_model.keys()).index(model_id)
            self.write_lock.release()
        else:
            print(f"{model_id} not found in gen_model.keys()")

    def pop_default_model(self):
        if len(self.default_model) > 1:
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.pop(-1)
            self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
            self.write_lock.release()

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        tries = 3*len(self.gen_model.keys())
        for attempt in range(tries):
            try:
                self.write_lock.acquire()
                token_use = self.token_count(kwargs["contents"])
                if self.gen_rpm > self.min_rpm and token_use <= self.token_quota and self.token_quota > self.min_tpm:
                    self.token_quota -= token_use
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, errors.APIError):
                    is_retry = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if api_error.code == 400:
                        print(f"retriable.api_error: token limit exceeded ({token_use})")
                    else:
                        print(f"retriable.api_error({api_error.code}): {str(api_error)}")
                    if not is_retry or attempt == tries:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                print(f"retriable.exception: {str(e)}")
                self.on_error(kwargs)
            finally:
                self.write_lock.release()

    def on_error(self, kwargs):
        self.generation_fail()
        kwargs["model"] = self(Api.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def generation_fail(self):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("Api.generation_fail.next_model: model is now", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.update_quota()

    def refill_rpm(self):
        self.running = False
        self.update_quota()
        print("Api.refill_rpm", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.update_quota()
        print("Api.zero_error: model is now", list(self.gen_model.keys())[self.m_id])

    def update_quota(self):
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.args.API_LIMIT]
        self.token_quota = list(self.gen_model.values())[self.m_id].tpm[self.args.API_LIMIT]*1_000_000

    def token_count(self, expr: str | list):
        count = self.args.CLIENT.models.count_tokens(
            model=self(Api.Model.GEN),
            contents=json.dumps(expr) if isinstance(expr, str) else str(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content) # type: ignore

Skipping Laminar.initialize()


In [3]:
# Define the embedding function.
api = NewType("api", Api) # type: ignore (forward-decl)
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Api.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task)) # type: ignore
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Api.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Api.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

## Set Gemini API Limit

In [4]:
# Instantiate the api-helper with usage limit => FREE.
# Optional: Set limit here to one of [FREE,TIER_1,TIER_2,TIER_3]
api = Api(with_limit=Api.Limit.FREE, default_model="gemini-2.5-flash")
# Export api environment for agent.
os.environ["API_LIMIT"]=str(api.args.API_LIMIT)
os.environ["GEN_DEFAULT"]=api.args.GEN_DEFAULT
# Cleanup old vector_db instances.
!rm -rf vector_db

# Gemini Baseline Check

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.args.CLIENT.chats.create(
    model=api(Api.Model.GEN),
    config=config_with_search,
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

The stock market is a global network of exchanges and marketplaces where individuals and institutions buy and sell ownership stakes in publicly traded companies. It serves as a crucial mechanism for companies to raise capital and for investors to grow their wealth.

**What is the Stock Market?**
At its core, the stock market is an aggregation of buyers and sellers of stocks, also known as shares or equities, which represent fractional ownership claims on businesses. These securities can be listed on public stock exchanges or traded privately. The terms "stock market" and "stock exchange" are often used interchangeably, but the stock market encompasses all individual stock exchanges.

**How the Stock Market Works:**
The stock market operates through a network of exchanges, such as the New York Stock Exchange (NYSE) and Nasdaq. It facilitates two main types of markets:
*   **Primary Market:** This is where new stocks are initially issued through a process called an Initial Public Offering (IPO). Companies sell shares directly to investors to raise capital for operations or expansion.
*   **Secondary Market:** After an IPO, shares enter the secondary market, which is essentially the stock exchange, where most daily trading occurs. Here, investors buy and sell existing shares among themselves, with the company no longer directly involved in these transactions.

Stock prices are primarily determined by the forces of supply and demand. If demand for a stock is high and the supply of available shares is low, the price tends to rise. Conversely, if many shareholders sell their shares, increasing supply, the price may fall. This process, where buyers and sellers negotiate prices, is known as price discovery. Over the long term, a company's profitability and earning power significantly influence the demand for its stock and, consequently, its price.

**Purpose of the Stock Market:**
The stock market serves two vital purposes:
1.  **Capital Generation for Companies:** It allows companies to raise significant capital from the public by selling shares, which they can then use to fund and expand their businesses without incurring debt.
2.  **Investment Opportunities for Individuals:** It provides investors with the opportunity to own a share in a company's profits and potentially grow their wealth. Investors can profit through capital gains (selling stock for more than they paid) or through dividends (a share of the company's profits paid out to shareholders).

**Key Participants:**
Various participants contribute to the functioning of stock markets:
*   **Individual (Retail) Investors:** Small investors who buy and sell stocks.
*   **Institutional Investors:** Large entities like pension funds, insurance companies, mutual funds, and hedge funds that make significant trades.
*   **Traders:** Individuals who buy and sell stocks over short time horizons, aiming to profit from small price fluctuations.
*   **Stockbrokers:** Professionals who execute buy and sell orders on behalf of investors.
*   **Stock Exchanges:** Provide the infrastructure for trades, maintain orderly markets, ensure regulatory compliance, and disseminate real-time price information.

**Factors Influencing Stock Market Fluctuations:**
Stock market movements are influenced by a range of factors, including:
*   **Macroeconomic Indicators:** Interest rates, inflation, and GDP growth.
*   **Company-Specific News:** Earnings reports, product launches, and leadership changes.
*   **Market Sentiment:** Investor psychology and overall confidence.
*   **Political Events and Geopolitical Tensions:** These can also play significant roles.

**Types of Investments in the Stock Market:**
Beyond individual stocks, the stock market offers various investment vehicles:
*   **Stocks (Equities):** Represent direct ownership in a company.
    *   **Common Stock:** Gives shareholders voting rights and potential for higher returns, but also higher risk.
    *   **Preferred Stock:** Often resembles bonds with fixed dividends and preference over common shareholders in receiving payments if the company dissolves.
    *   Stocks can also be categorized by market capitalization (large-cap, mid-cap, small-cap), geographic location (domestic, international), dividend payment (dividend, non-dividend), and industry characteristics (cyclical, non-cyclical, ESG).
*   **Bonds:** Fixed-income securities offered by governments and businesses, representing a loan made by an investor to a borrower.
*   **Mutual Funds:** Professionally managed pools of money from multiple investors, invested in a diversified portfolio of stocks, bonds, or other securities.
*   **Exchange-Traded Funds (ETFs):** Similar to mutual funds but trade like individual stocks on an exchange.
*   **Derivatives:** Financial contracts whose value is derived from an underlying asset, such as options.
*   **Initial Public Offerings (IPOs):** The first sale of stock by a private company to the public.

In [6]:
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Amazon.com Inc. (NASDAQ: AMZN) is a multinational technology company that has significantly diversified its operations beyond its origins as an online bookstore. It is now a major player in e-commerce, cloud computing (through Amazon Web Services, or AWS), online advertising, digital streaming, and artificial intelligence.

Here's a snapshot of AMZN stock and related information:

**Current Stock Price and Market Data (as of November 28, 2025):**
*   **Stock Price:** Around $230.05 to $233.22 USD.
*   **Market Capitalization:** Approximately $2.49 trillion.
*   **Price-to-Earnings (P/E) Ratio:** 32.37.
*   **52-Week Range:** The stock has fluctuated between a low of $161.38 and a high of $258.60 over the past year.
*   **Average Daily Volume:** Around 51.02 million shares.

**Recent Performance:**
*   As of November 28, 2025, the stock was trading slightly above its daily low and off its daily high.
*   AMZN stock has seen a 7.80% increase compared to the previous week and a 0.67% rise over the last month.
*   Over the last year, Amazon.com, Inc. has shown a 12.68% increase. Another source indicates a 12.18% change over the past year.

**Company Overview:**
Founded by Jeff Bezos in 1994, Amazon initially focused on selling books online before expanding into a vast array of products and services, earning it the moniker "The Everything Store". Today, Amazon is the world's largest online retailer and marketplace, a leading smart speaker provider, and a dominant force in cloud computing through AWS. AWS is a significant contributor to Amazon's operating profits. The company also offers digital content subscriptions (Amazon Prime), online advertising, and manufactures electronic devices like Kindle, Fire, and Echo.

**Financial Highlights:**
*   In 2024, Amazon's revenue was $637.96 billion, an 11% increase year-over-year from $575 billion in 2023. Operating income improved by 86% year-over-year to $68.6 billion.
*   For the third quarter ended September 30, 2025, net sales increased 13% to $180.2 billion.
*   The company's earnings per share (EPS) for the latest quarter was $1.95, exceeding an estimation of $1.57.

**Analyst Sentiment and Outlook:**
*   The average analyst rating for Amazon stock is "Strong Buy".
*   Analysts have a consensus price target ranging from $280.47 to $295.78, forecasting a potential increase of 20.80% to 26.78% over the next year from current prices. The highest target is $360.00, and the lowest is $250.00.
*   A significant number of analysts (between 47 and 78) cover AMZN, with a high "Buy % Consensus".

**Recent News and Developments:**
*   Amazon continues to invest heavily in AI and supercomputing infrastructure. Recent announcements include plans to invest $15 billion in Northern Indiana and $3 billion in Mississippi for new data center campuses.
*   The company is also pushing its in-house AI development system, Kiro, encouraging engineers to adopt it over third-party tools.
*   However, Amazon is facing scrutiny, with over 1,000 employees signing an open letter expressing concerns about the company's rapid AI development impacting climate goals and potentially leading to job displacement.
*   Amazon announced its next earnings report is expected on January 29, 2026.

**Important Note:** Investing in the stock market involves risks, and past performance is not indicative of future results. It's always recommended to conduct thorough research and consider consulting with a financial advisor before making any investment decisions.

In [7]:
response = chat.send_message('''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Amazon.com Inc. (NASDAQ: AMZN) is currently trading around **$233.22 USD** as of November 28, 2025, reflecting a 1.77% increase in the past 24 hours. The stock's market capitalization stands at approximately $2.49 trillion.

**Short-Term Trends:**
In the short term, AMZN has shown some mixed signals. The stock has risen by 7.80% compared to the previous week and 0.67% over the last month. However, it experienced a -1.85% decline over the past two weeks.

Technically, Amazon stock holds buy signals from both short and long-term Moving Averages, suggesting a positive forecast. However, a general sell signal is present due to the long-term average being above the short-term average. Support levels are identified around $225.50 and $229.72.

More specifically, as of November 28, 2025, AMZN shares closed at $229.20, trading below its 20-day average ($236.16) but above its 50-day average ($227.53). This indicates short-term downside pressure but a long-term bullish structure. Momentum signals are mixed, with the MACD showing a bearish bias, the RSI at 48.69 (neutral), and Bull/Bear Power at 2.05, suggesting buyers still dominate despite some selling pressure. The expected five-session trading range is between $224 and $235, with an over 80% probability of a move higher. Key resistance is at $236.90 and support at $227.50.

**Bullish vs. Bearish Predictions:**

**Bullish Outlook:**
The overall sentiment from analysts for AMZN stock is overwhelmingly bullish. The consensus rating from 43 to 61 Wall Street analysts is a "Strong Buy" or "Buy."

*   **Price Targets:** Analysts have set an average 12-month price target ranging from $280.47 to $296.42. This represents a potential increase of 20.80% to 29.35% from current prices. The highest price targets range from $335.00 to $360.00.
*   **Key Drivers:**
    *   **AWS Growth:** Amazon Web Services (AWS) is a significant contributor to Amazon's operating profits and continues to show strong growth. AWS revenue accelerated to 20.2% year-over-year growth in Q3 2025, with a $200 billion backlog. Analysts project AWS revenue to reach $128.1 billion in 2025 and $348.5 billion in 2030.
    *   **AI Demand:** Amazon is well-positioned to benefit from surging AI demand and cloud infrastructure growth, with significant investments in AI and supercomputing infrastructure. AWS's Trainium chips business is already at a multi-billion dollar run rate.
    *   **Operational Efficiency:** Cost savings through automation in fulfillment centers are expected to improve operating margins, with estimated annual savings of $2-$4 billion by 2027.
    *   **Retail and Advertising:** Retail-related revenue, including online stores and third-party seller contributions, continues to grow, and the expansion of digital advertising also supports the bullish case.

**Bearish Outlook:**
While the dominant sentiment is bullish, some potential concerns and bearish arguments exist:

*   **Volatility:** Despite positive analyst ratings, the stock has faced volatility, which can pose risks for investors seeking stable returns.
*   **Technical Weaknesses:** Some technical indicators show a bearish bias in the daily MACD and a weak overall trend (low ADX).
*   **Macroeconomic Pressures:** Intense competition, difficulties in international expansion, and macroeconomic pressures affecting consumer spending could contribute to a deterioration of the company's financial outlook.
*   **Employee Concerns:** Amazon is facing scrutiny from over 1,000 employees regarding the rapid AI development impacting climate goals and potentially leading to job displacement.

In summary, while AMZN has experienced some short-term fluctuations, the overarching analyst sentiment remains strongly bullish, driven by the robust performance of AWS, significant investments in AI, and ongoing operational efficiencies.

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

Metro-Goldwyn-Mayer (MGM) Studios does not have its own stock ticker symbol because it is no longer a publicly traded company. Amazon acquired MGM in March 2022, and it now operates as a wholly owned subsidiary under the umbrella of Amazon MGM Studios.

Therefore, if you are interested in the financial performance of the company that owns MGM Studios, you would look at the stock performance of **Amazon.com Inc. (NASDAQ: AMZN)**.

It is important not to confuse MGM Studios with MGM Resorts International, which is a separate, publicly traded company focused on hospitality and entertainment (casinos and resorts) and trades under the ticker symbol **NYSE: MGM**.

In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Here is the last available open, close, high, and low data for AMZN stock as of November 28, 2025:

*   **Open:** $231.24
*   **Close:** $233.22
*   **High:** $233.29
*   **Low:** $230.22

In [10]:
response = chat.send_message('''What is AMZN open,close,high,low data for the past month? 
Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here is the open, close, high, and low data for AMZN stock for the past month, from October 28, 2025, to November 28, 2025:

| Date         | Open (USD) | Close (USD) | High (USD) | Low (USD) |
| :----------- | :--------- | :---------- | :--------- | :-------- |
| Nov 28, 2025 | $231.24    | $233.22     | $233.29    | $230.22   |
| Nov 27, 2025 | $229.21    | $229.21     | $230.22    | $228.20   |
| Nov 26, 2025 | $230.74    | $229.16     | $231.7474  | $228.77   |
| Nov 25, 2025 | $226.38    | $229.67     | $230.52    | $223.80   |
| Nov 24, 2025 | $222.555   | $226.28     | $227.33    | $222.27   |
| Nov 21, 2025 | $216.345   | $220.69     | $222.21    | $215.18   |
| Nov 20, 2025 | $227.05    | $217.14     | $227.41    | $216.74   |
| Nov 19, 2025 | $223.735   | $222.69     | $223.735   | $222.55   |
| Nov 18, 2025 | $228.10    | $222.55     | $230.20    | $222.55   |
| Nov 17, 2025 | $233.25    | $232.87     | $234.60    | $232.87   |
| Nov 14, 2025 | $235.06    | $234.69     | $238.73    | $232.89   |
| Nov 13, 2025 | $243.05    | $237.58     | $243.75    | $236.50   |
| Nov 12, 2025 | $250.24    | $244.25     | $250.37    | $243.75   |
| Nov 11, 2025 | $248.41    | $249.10     | $249.75    | $247.23   |
| Nov 10, 2025 | $248.34    | $248.40     | $251.75    | $245.59   |
| Nov 07, 2025 | $250.00    | $249.00     | $251.00    | $248.00   |
| Nov 06, 2025 | $252.00    | $250.00     | $253.00    | $249.00   |
| Nov 05, 2025 | $253.00    | $252.00     | $254.00    | $251.00   |
| Nov 04, 2025 | $254.00    | $253.00     | $255.00    | $252.00   |
| Nov 03, 2025 | $254.00    | $254.00     | $255.00    | $253.00   |
| Oct 31, 2025 | $244.22    | $244.22     | $244.22    | $244.22   |
| Oct 30, 2025 | $240.00    | $242.00     | $243.00    | $239.00   |
| Oct 29, 2025 | $235.00    | $238.00     | $239.00    | $234.00   |
| Oct 28, 2025 | $231.49    | $229.25     | $231.49    | $226.21   |

Please note that some data points may vary slightly across different financial sources due to reporting times or adjustments. The closing price for Amazon (AMZN) on October 31, 2025, was $244.22, and it was up 12.4% for the month. The all-time high Amazon stock closing price was $254.00 on November 03, 2025.

# Previously on Kaggle: StockChat 1.0

## Validation BaseModels

In [11]:
# Validation BaseModels in pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Api.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    V_POS = "very positive"
    POSITIVE = "positive"
    NEUTRAL_P = "neutral/positive"
    NEUTRAL_SP = "neutral/slightly positive"
    NEUTRAL = "neutral"
    NEUTRAL_SN = "neutral/slightly negative"
    NEUTRAL_N = "neutral/negative"
    MIXED = "mixed"
    NEGATIVE = "negative"
    V_NEG = "very negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    BULLN = "cautiously bullish"
    HOLD = "hold"
    BEARN = "cautiously bearish"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        datetime_now = datetime.now(self.tz())
        self_ts = parse(self.timestamp)
        # Re-generate events for a new day.
        if datetime_now.day > self_ts.day:
            return True
        # No updates on holidays or when generated after post_close.
        if self.is_holiday or self_ts.time() >= parse(self.post_close).time():
            return False
        # Compare current time to generated event times.
        for event in [self.pre_open,self.reg_open,self.reg_close]:
            if datetime_now.time() > parse(event).time():
                return True
        # Current time is before pre_open.
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 or date_now.weekday() == 1 and date_gen.weekday() <= 4: # 0=monday, 4=friday
            return value # pass: generated thurs/friday on a monday/tues
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() <= date_now.weekday()-1:
            return value # pass: generated yesterday/prior on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() <= 4:
            return value # pass: generated thurs/friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Contents Memory

In [12]:
# A contents-memory object.
class Memory:
    def __init__(self):
        self.system = f"""Give a concise, and detailed summary. Use information that you learn from the API responses.
        Use your tools and function calls according to the rules. Convert any all-upper case identifiers
        to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
        Convert timestamps according to the rules before including them. Think step by step.
        """
        self.revery = {}
        self.contents = []
        self.prompt = None
        self.summary = None
        self.response = None
    
    def set_prompt(self, prompt):
        self.prompt = f"""
        The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
        
        {prompt}
        """
        self.contents = [types.Content(role="user", parts=[types.Part(text=self.prompt)])]

    def set_reason(self, step):
        # Append the model's reasoning part.
        self.contents.append(types.Content(role="model", parts=[types.Part(thought=True,text=step)]))

    def append_code(self, prompt, code_response_parts):
        subroutine_content = [types.Content(role="user", parts=[types.Part(text=prompt)]),
                              types.Content(role="model", parts=code_response_parts)]
        # Append the model's generated code and execution result.
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = { 
            "contents": subroutine_content
        }

    def update_contents(self, function_call, api_response_part):
        # Append the model's function call part.
        self.contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
        # Append the api response part.
        self.contents.append(types.Content(role="user", parts=[api_response_part]))

    def set_summary(self, summary):
        self.summary = summary
        self.contents.append(types.Content(role="model", parts=[types.Part(text=summary)]))
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = {
            "prompt": self.prompt, 
            "summary": self.summary, 
            "contents": self.contents
        }
        self.contents = []

memory = Memory()

## Retrieval-Augmented Generation

In [13]:
# Define tool: retrieval-augmented generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}
    holidays: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn,  # type: ignore
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id
        self.set_holidays("US", ["09-01-2025","10-13-2025","11-11-2025","11-27-2025","12-25-2025"])
        #self.generated_events("US")

    def set_holidays(self, exchange_code: str, holidays: list):
        self.holidays[exchange_code] = [datetime.strptime(h, "%m-%d-%Y").date() for h in holidays]

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_lists[with_query] = ast.literal_eval(data.parts[-1].text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_codes = ast.literal_eval(data.parts[-1].text.strip("\\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def get_event_date(self, event_t: str, exchange_code: str, event: MarketEvent):
        current_dt_str = datetime.now(GeneratedEvent.tz()).strftime('%c')
        current_dt = datetime.strptime(current_dt_str, "%a %b %d %H:%M:%S %Y")
        current_t_str = datetime.now(GeneratedEvent.tz()).strftime('%H:%M:%S')
        current_t = datetime.strptime(current_t_str, "%H:%M:%S").time()
        event_time = parse(event_t).time()
        gen_datetime = None
        if event is MarketEvent.LAST_CLOSE:
            last_close_day = current_dt.date() - timedelta(days=0 if current_t > event_time else 1)
            # Loop backwards to find the last valid trading day (not a weekend or holiday).
            while last_close_day.weekday() >= 5 or last_close_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                last_close_day -= timedelta(days=1)
            # Combine the date and time.
            gen_datetime = datetime.combine(last_close_day, event_time)
        else:
            next_event_day = current_dt.date() + timedelta(days=0 if current_t < event_time else 1)
            # Loop forward to find the next valid trading day (not a weekend or holiday).
            while next_event_day.weekday() >= 5 or next_event_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                next_event_day += timedelta(days=1)
            # Combine date and time.
            gen_datetime = datetime.combine(next_event_day, event_time)
        # Format the result as requested.
        return gen_datetime.strftime('%a %b %d %X %Y')

    def generate_event(self, exchange_code: str, event: MarketEvent):
        if event is MarketEvent.LAST_CLOSE or event is MarketEvent.POST_CLOSE:
            prompt = f"""What is the closing time including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""What is the opening time {is_pre} pre_market hours."""
        elif event is MarketEvent.REG_CLOSE:
            prompt = f"""What is the closing time excluding post_market hours."""
        prompt = f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with the time in this format: '%H:%M:%S'.
            Omit all other chat and details. Do not use sentences."""
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        response = self.get_exchanges_csv(prompt).candidates[0].content
        try:
            if Api.Const.Stop() in f"{response.parts[-1].text}":
                self.generate_event_failed(progress, exchange_code, event)
            else:
                response = self.get_event_date(response.parts[-1].text, exchange_code, event)
                progress.update(1)
                return response
        except Exception as e:
            self.generate_event_failed(progress, exchange_code, event)

    def generate_event_failed(self, progress: tqdm, exchange_code: str, event: MarketEvent):
        progress.close()
        api.generation_fail()
        time.sleep(api.dt_between)
        return self.generate_event(exchange_code, event)

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        # Check for an existing GeneratedEvent object having updates.
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            event_obj = self.events[exchange_code]
            event_state = [(event_obj.pre_open, MarketEvent.PRE_OPEN),
                           (event_obj.reg_open, MarketEvent.REG_OPEN),
                           (event_obj.reg_close, MarketEvent.REG_CLOSE),
                           (event_obj.post_close, MarketEvent.POST_CLOSE)]
            # Need now in same format as generated.
            datetime_now = parse(datetime.now(event_obj.tz()).strftime('%c'))
            gen_ts = parse(event_obj.timestamp)
            # Re-generate events when day changes.
            if datetime_now.day > gen_ts.day:
                del self.events[exchange_code]
                return self.generated_events(exchange_code)
            # Update changed events on trading days.
            for e in event_state:
                if datetime_now > parse(e[0]):
                    event_dt = self.generate_event(exchange_code, e[1])
                    match e[1]:
                        case MarketEvent.PRE_OPEN:
                            event_obj.pre_open = event_dt
                        case MarketEvent.REG_OPEN:
                            event_obj.reg_open = event_dt
                        case MarketEvent.REG_CLOSE:
                            event_obj.reg_close = event_dt
                        case MarketEvent.POST_CLOSE:
                            event_obj.post_close = event_dt
            event_obj.timestamp = datetime.now(event_obj.tz()).strftime('%c')
            self.events[exchange_code] = event_obj
        # Generate events for an exchange code not in cache.
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE),
                is_holiday=datetime.now().date() in self.holidays[exchange_code])
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Api.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
        # Generate the response.
        response = api.retriable(
            self.client.models.generate_content,
            model=api(Api.Model.GEN),
            config=self.config_temp,
            contents=prompt)
        # Check for generated code and store in memory.
        content = response.candidates[0].content
        if len(content.parts) > 1 and content.parts[0].executable_code:
            memory.append_code(prompt, content.parts)
        return response

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wiki Grounding

In [14]:
# Define tool: wiki-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Api.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        page_html = Api.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(page_html)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return Api.Const.Stop()

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

## Search Grounding

In [15]:
# Define tool: search-grounding generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides answers to questions about {topic}.
        Do not discuss alternative topics of interest. Do not discuss similar topics.
        You will provide answers that discuss only {topic}. 
        You may discuss the owner or parent of {topic} when no other answer is possible.
        Otherwise respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Api.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return Api.Const.Stop() # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches[0]
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest Grounding

In [16]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",50
    POLY = "polygon.io",4 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [17]:
# Define tool: rest-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(Api.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = Api.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.92
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return Api.Const.Stop()

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Api.Const.MetricBatch()):
            batch = metric[i:i + Api.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Api.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Api.Const.SeriesBatch()):
                        batch = v[i:i + Api.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return Api.Const.Stop()

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return Api.Const.Stop()

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

## Callable Functions

In [18]:
# Callable functions in openapi schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """A ticker symbol to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It provides the last trading days' data after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It includes 
                   the last trading days' data after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be included and equal to 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """This must be included. May be one of asc or desc. asc will sort by timestmap in 
                                  ascending order. desc will sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be included and equal to 1000 unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["limit", "ticker", "published_utc.gte", "published_utc.lte", "order", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

In [19]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [20]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.args.CLIENT, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.args.CLIENT, tool_rag)
tool_ground = SearchGroundingGenerator(api.args.CLIENT, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]


## Function Calling Expert

In [21]:
# Implement the callable functions and function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != Api.Const.Stop():
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return Api.Const.Stop()
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [22]:
# Implement the function calling expert.
# Define the system prompt.
instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

def get_response():
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    memory.response = api.retriable(
        api.args.CLIENT.models.generate_content,
        model=api(Api.Model.GEN),
        config=config_fncall,
        contents=memory.contents)

def retry_last_send():
    api.generation_fail()
    time.sleep(api.dt_between)
    get_response()

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    memory.set_prompt(prompt)
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    get_response()
    while function_calling_in_process:
        try:
            response_parts = memory.response.candidates[0].content.parts
            # A summary response never includes function calls.
            if not any(part.function_call for part in response_parts):
                memory.set_summary("\n".join(e.text for e in response_parts))
                function_calling_in_process = False
                break # The function calling chain is complete.
            else:
                # A part can be a function call or reasoning-step.
                for part in response_parts:
                    if function_call := part.function_call:
                        # Extract the function call.
                        fn_name = function_call.name
                        #display(Markdown("#### Predicted function name"))
                        #print(fn_name, "\n")
                        # Extract the function call arguments.
                        fn_args = {key: value for key, value in function_call.args.items()}
                        #display(Markdown("#### Predicted function arguments"))
                        #print(fn_args, "\n")
                        # Call the predicted function.
                        print("send_message: get function response")
                        api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                        #display(Markdown("#### API response"))
                        #print(api_response[:500], "...", "\n")
                        # Create an API response part.
                        api_response_part = types.Part.from_function_response(
                            name=fn_name,
                            response={"content": api_response},
                        )
                        memory.update_contents(function_call, api_response_part)
                    else:
                        #display(Markdown("#### Natural language reasoning step"))
                        #print(part.text)
                        memory.set_reason(part.text)
                print("send_message: updating state")
                get_response() # Send the updated prompt.
                print("send_message: got a response")
        except Exception as e:
            if isinstance(response_parts, list):
                print("send_message: generated wrong function arguments")
            retry_last_send()
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(memory.summary))

# RAG Baseline Check

In [23]:
response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.candidates[0].content.parts[-1].text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.candidates[0].content.parts[-1].text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    The fields pre_market and post_market both represent extended operating hours.

    The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

    Weekdays are: Mon, Tue, Wed, Thu, Fri.
    On weekdays all exchanges open after pre-market and regular hours.
    On weekdays all exchanges close after regular and post-market hours.
    
    Weekends are: Sat, Sun.
    Always exclude weekends from exchange operating hours.
    A list of holidays in date format mm-dd-yyyy: {tool_rag.holidays["US"]}
    Always exclude holidays from exchange operating hours.
    When the answer is a holiday use the prior weekday for close.
    When the answer is a holiday use the next weekday for open.
    
    Consider the US exchange's operating hours.
    Provide the most recent weekday's close including post_market hours.
    
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.candidates[0].content.parts[-1].text)

```
{
    "VN": "Vietnam exchanges including HOSE, HNX and UPCOM",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "US": "US exchanges (NYSE, Nasdaq)",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "QA": "QATAR EXCHANGE",
    "BA": "BOLSA DE COMERCIO DE BUENOS AIRES",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
    "PR": "PRAGUE STOCK EXCHANGE",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "CA": "Egyptian Stock Exchange",
    "AX": "ASX - ALL MARKETS",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "DB": "DUBAI FINANCIAL MARKET",
    "PM": "Philippine Stock Exchange",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "ST": "NASDAQ OMX NORDIC STOCKHOLM",
    "DU": "BOERSE DUESSELDORF",
    "TL": "NASDAQ OMX TALLINN",
    "AT": "ATHENS EXCHANGE S.A. CASH MARKET",
    "SW": "SWISS EXCHANGE",
    "LS": "NYSE EURONEXT - EURONEXT LISBON",
    "SI": "SINGAPORE EXCHANGE",
    "RG": "NASDAQ OMX RIGA",
    "CR": "CARACAS STOCK EXCHANGE"

# SC1 Baseline Check

In [24]:
# Wait 59s for rate-limits to reset on FREE-tier.
if api.args.API_LIMIT is Api.Limit.FREE.value:
    print("Gemini API limit is FREE. Waiting 59s...")
    time.sleep(59)

Gemini API limit is FREE. Waiting 59s...
Api.refill_rpm 10


In [25]:
send_message("What is the current session for US exchanges?")

send_message: get function response


Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:01<00:00,  1.88s/it]
Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The current session for US exchanges is closed.

In [26]:
send_message("What is the US market status?")

Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The US market is currently closed. The status was last updated on Sat Nov 29 14:09:46 2025 Eastern Time.

In [27]:
send_message("When was the last US market close?")

send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The last US market close was on Friday, November 28, 2025, at 8:00:00 PM.

In [28]:
send_message("What is Apple's stock ticker?")

send_message: get function response


Score similarity to query: 100%|██████████| 10/10 [00:00<00:00, 12.45it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The stock ticker for Apple is AAPL.

In [29]:
send_message("What is the current price of Amazon stock? Display the result as a json string in markdown.")

send_message: get function response


Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  5.57it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response
send_message: get function response


Generate quote embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The current price of Amazon stock (AMZN) is $233.22 as of Fri Nov 28 2025 16:00:00. The price changed by $4.06, which is a 1.7717% change. The high price of the day was $233.285, and the low price was $230.22. The opening price was $231.24, and the previous close price was $229.16.

```json
{
"symbol": "AMZN",
"current_price": 233.22,
"change": 4.06,
"percent_change": 1.7717,
"high_price": 233.285,
"low_price": 230.22,
"open_price": 231.24,
"previous_close_price": 229.16,
"timestamp": "Fri Nov 28 2025 16:00:00"
}
```

In [30]:
send_message("""Show me Apple's basic financials and help me understand key performance metrics. 
How has the stock performed?""")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's an overview of Apple's financial health based on the data from September 27, 2025:

**Key Financial Metrics:**
*   **Revenue:**
    *   Revenue Employee Annual: $2.5376 million
    *   Revenue Per Share Annual: $27.7354
    *   Revenue Growth (3Y): 1.81%
    *   Revenue Growth (5Y): 8.68%
*   **Profitability:**
    *   Gross Margin: 46.91%
    *   Operating Margin: 31.97%
    *   Net Profit Margin: 26.92%
    *   EBITD Per Share Annual: $9.6255
    *   EPS Annual: $7.465
*   **Asset Management:**
    *   Asset Turnover TTM: 1.2186
    *   Inventory Turnover TTM: 33.9834
*   **Debt & Equity:**
    *   Long Term Debt/Equity Annual: 1.0623
    *   Total Debt/Equity Annual: 1.338

**Stock Performance:**
*   52 Week High: $280.38 on 2025-11-25
*   52 Week Low: $169.2101 on 2025-04-08
*   52 Week Price Return Daily: 18.6293%
*   Year-to-Date Price Return Daily: 11.3529%
*   Month-to-Date Price Return Daily: 3.1364%
*   5 Day Price Return Daily: 2.711%

**Valuation Ratios:**
*   PE TTM: 36.7859
*   Forward PE: 33.8620
*   Price-to-Book (PB): 55.8825
*   Price-to-Sales (PS): 9.9009
*   PEG TTM: 1.6255

**Additional Insights:**

*   **Margins:** Apple maintains strong margins, indicating efficient operations and a strong brand.
*   **Debt:** The debt-to-equity ratio suggests a moderate level of leverage.
*   **Growth:** While revenue growth over the past three years has been modest, the five-year growth rate is more robust.
*   **Stock Performance:** The stock has shown positive price returns over the past year and year-to-date.

These metrics provide a snapshot of Apple's financial standing and stock performance as of the provided date.


In [31]:
send_message("I need Apple's daily candlestick from 2025-05-05")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

On 2025-05-05, Apple's stock (AAPL) had the following daily candlestick data: open price was 203.1, high price was 204.1, low price was 198.21, and close price was 198.89. The trading volume was 69018452. The pre-market price was 205.0, and the after-hours price was 198.6.


In [32]:
send_message("Tell me who are Apple's peers?")

send_message: get function response


Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 27.68it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.28it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.77it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.22it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 27.75it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.91it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 11.00it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.56it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 11.17it/s]

send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response


#### Natural language response

Apple's peers include DELL (DELL TECHNOLOGIES -C), WDC (WESTERN DIGITAL CORP), SNDK (SANDISK CORP), PSTG (PURE STORAGE INC - CLASS A), HPE (HEWLETT PACKARD ENTERPRISE), HPQ (HP INC), NTAP (NETAPP INC), SMCI (SUPER MICRO COMPUTER INC), IONQ (IONQ INC), QUBT (QUANTUM COMPUTING INC), and CMPO (COMPOSECURE INC-A).


In [33]:
send_message("Tell me who are Amazon's peers?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  2.65it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 11.96it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.95it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 28.64it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 62.48it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 12.13it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.16it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  6.15it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.07it/

send_message: updating state
send_message: got a response


#### Natural language response

Amazon's peers are: COUPANG INC, EBAY INC, DILLARDS INC-CL A, OLLIE'S BARGAIN OUTLET HOLDI, MACY'S INC, ETSY INC, KOHLS CORP, PATTERN GROUP INC-CL A, SAVERS VALUE VILLAGE INC, GROUPON INC.

In [34]:
send_message("""Locate Apple's stock ticker, then download recommendation trends of all Apple's peers by sub-industry, 
and then finally compare them.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


...

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's a comparison of the latest recommendation trends (as of November 2025) for Apple's peers in the sub-industry:

*   **DELL (DELL TECHNOLOGIES -C):** 8 Strong Buy, 18 Buy, 6 Hold, 0 Sell, 0 Strong Sell
*   **WDC (WESTERN DIGITAL CORP):** 6 Strong Buy, 19 Buy, 7 Hold, 0 Sell, 0 Strong Sell
*   **SNDK (SANDISK CORP):** 7 Strong Buy, 10 Buy, 8 Hold, 0 Sell, 0 Strong Sell
*   **PSTG (PURE STORAGE INC - CLASS A):** 6 Strong Buy, 13 Buy, 7 Hold, 1 Sell, 0 Strong Sell
*   **HPE (HEWLETT PACKARD ENTERPRISE):** 6 Strong Buy, 8 Buy, 12 Hold, 0 Sell, 0 Strong Sell
*   **HPQ (HP INC):** 2 Strong Buy, 3 Buy, 16 Hold, 1 Sell, 0 Strong Sell
*   **NTAP (NETAPP INC):** 3 Strong Buy, 9 Buy, 15 Hold, 0 Sell, 0 Strong Sell
*   **SMCI (SUPER MICRO COMPUTER INC):** 2 Strong Buy, 10 Buy, 11 Hold, 3 Sell, 0 Strong Sell
*   **IONQ (IONQ INC):** 2 Strong Buy, 10 Buy, 3 Hold, 0 Sell, 0 Strong Sell
*   **QUBT (QUANTUM COMPUTING INC):** 2 Strong Buy, 5 Buy, 2 Hold, 0 Sell, 0 Strong Sell
*   **CMPO (COMPOSECURE INC-A):** 2 Strong Buy, 8 Buy, 1 Hold, 1 Sell, 0 Strong Sell

In [35]:
send_message("""Tell me Amazon's current share price and provide candlestick data for the past month. 
Sort the data in descending order by date. Format the prices consistently as currency. 
Round prices to two decimal places. 
Present the data with multiple columns for display in markdown. 
Discuss and provide details about any patterns you notice in the price data. 
Correlate recent patterns with news over the same date range.""")

Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

Amazon's current share price and the candlestick data for the past month are detailed below.

The current share price for **Amazon (AMZN)** is **$233.22**. This represents a change of **$4.06** (+1.77%) from the previous close of $229.16.

### Amazon (AMZN) Candlestick Data (October 28, 2025 - November 28, 2025)

| Date | Open | High | Low | Close | Volume |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **Fri Nov 28 2025** | $231.24 | $233.29 | $230.22 | **$233.22** | 20,250,425 |
| **Wed Nov 26 2025** | $230.74 | $231.75 | $228.77 | **$229.16** | 38,497,719 |
| **Tue Nov 25 2025** | $226.38 | $230.52 | $223.80 | **$229.67** | 39,379,339 |
| **Mon Nov 24 2025** | $222.56 | $227.33 | $222.27 | **$226.28** | 54,318,223 |
| **Fri Nov 21 2025** | $216.35 | $222.21 | $215.18 | **$220.69** | 68,490,453 |
| **Thu Nov 20 2025** | $227.05 | $227.41 | $216.74 | **$217.14** | 50,308,862 |
| **Wed Nov 19 2025** | $223.74 | $223.74 | $218.52 | **$222.69** | 58,335,353 |
| **Tue Nov 18 2025** | $228.10 | $230.20 | $222.42 | **$222.55** | 60,608,442 |
| **Mon Nov 17 2025** | $233.25 | $234.60 | $229.19 | **$232.87** | 59,918,908 |
| **Fri Nov 14 2025** | $235.06 | $238.73 | $232.89 | **$234.69** | 38,956,619 |
| **Thu Nov 13 2025** | $243.05 | $243.75 | $236.50 | **$237.58** | 41,401,638 |
| **Wed Nov 12 2025** | $250.24 | $250.37 | $243.75 | **$244.20** | 31,190,063 |
| **Tue Nov 11 2025** | $248.41 | $249.75 | $247.23 | **$249.10** | 23,563,960 |
| **Mon Nov 10 2025** | $248.34 | $251.75 | $245.59 | **$248.40** | 36,476,474 |
| **Fri Nov 7 2025** | $242.90 | $244.90 | $238.49 | **$244.41** | 46,374,294 |
| **Thu Nov 6 2025** | $249.16 | $250.38 | $242.17 | **$243.04** | 46,004,201 |
| **Wed Nov 5 2025** | $249.03 | $251.00 | $246.16 | **$250.20** | 40,610,602 |
| **Tue Nov 4 2025** | $250.38 | $257.01 | $248.66 | **$249.32** | 51,546,311 |
| **Mon Nov 3 2025** | $255.36 | $258.60 | $252.90 | **$254.00** | 95,997,714 |
| **Fri Oct 31 2025** | $250.10 | $250.50 | $243.98 | **$244.22** | 166,340,683 |
| **Thu Oct 30 2025** | $227.06 | $228.44 | $222.75 | **$222.86** | 102,252,888 |
| **Wed Oct 29 2025** | $231.67 | $232.82 | $227.76 | **$230.30** | 52,035,936 |
| **Tue Oct 28 2025** | $228.22 | $231.49 | $226.21 | **$229.25** | 47,099,924 |

***

### Price Pattern Analysis and Correlation with News

The price data for Amazon (AMZN) over the past month reveals a period of significant volatility, characterized by a sharp surge followed by a notable correction, and then a period of consolidation.

#### 1. The Post-Earnings Surge (Late October - Early November)
*   **Pattern:** The stock experienced a massive upward gap and surge in the days following the last week of October. The price jumped from a close of **$229.25** on Tue Oct 28 to **$244.22** on Fri Oct 31, and then peaked at **$254.00** on Mon Nov 3. This is a clear **bullish breakout** driven by a major catalyst.
*   **News Correlation:** This surge is directly correlated with Amazon's strong Q3 earnings report and major AI-related news:
    *   **Thu Oct 30:** Amazon reported strong Q3 earnings, with AWS revenue growing 20% year-over-year. This news was the primary catalyst, as the stock surged **10%** on Fri Oct 31.
    *   **Mon Nov 3 - Tue Nov 4:** The stock continued its rally, fueled by the announcement of a massive **$38 billion cloud computing services deal with OpenAI** for AI chip access and infrastructure. This news was seen as a major win for AWS, solidifying its position in the AI race.

#### 2. The Correction (Mid-November)
*   **Pattern:** Following the peak of **$254.00** on Mon Nov 3, the stock entered a sharp decline, dropping to a low of **$217.14** on Thu Nov 20. This represents a drop of approximately **14.5%** in just over two weeks, officially entering a **correction** phase (a drop of 10% or more from a recent high).
*   **News Correlation:** This correction was driven by broader market skepticism and company-specific concerns:
    *   **Market Jitters:** Multiple articles in mid-November (e.g., Nov 5, Nov 7, Nov 13, Nov 18) discussed a **tech stock selloff** and a **market correction** driven by investor concerns about the sustainability of massive AI capital expenditures (CapEx) by tech giants like Amazon. Analysts warned of potential overspending and a lack of clear, immediate returns on these huge investments.
    *   **Negative Sentiment:** News on **Nov 18** and **Nov 19** highlighted Amazon selling stakes in AI/Quantum computing stocks (IonQ and AMD) and a report on a projected **45% dip in Amazon's free cash flow (FCF)** in 2025 due to aggressive AI investments, which likely spooked investors focused on near-term profitability.

#### 3. The Consolidation and Recovery (Late November)
*   **Pattern:** After hitting the low on Thu Nov 20, the price stabilized and began a gradual, choppy recovery, moving from **$217.14** to **$233.22** by Fri Nov 28. This is a **rebound** from the correction low, with the price consolidating in the low $230s.
*   **News Correlation:** The recovery was supported by renewed positive sentiment and holiday shopping optimism:
    *   **Analyst Confidence:** An article on **Nov 27** noted that Amazon had entered a correction zone but analysts remained universally confident, viewing the pullback as a **buying opportunity** with high price targets.
    *   **Holiday Season:** News on **Nov 24** and **Nov 26** highlighted Amazon's strong positioning for the Black Friday/Cyber Monday shopping period, with positive consumer studies and expectations for a strong year-end rally in the retail sector.
    *   **Legal Victory:** A positive legal development on **Nov 27**, where a judge blocked New York's bid to regulate private-sector labor disputes, also contributed to the positive sentiment.

In [36]:
send_message("What is Apple's ticker overview")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response


#### Natural language response

Apple Inc. (AAPL) is a US-based stock in the stocks market, primarily traded on the XNAS exchange. The company has a market capitalization of $4,120,386,034,050.0005. Apple's main products are hardware and software, including the iPhone, Mac, iPad, and Watch. The company employs 166,000 people. Apple's branding assets include a logo available at https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_logo.svg and an icon at https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_icon.png.


In [37]:
send_message("What is Google's stock ticker symbol?")

send_message: get function response
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Google's stock is listed on the NASDAQ stock exchange under two ticker symbols: **GOOGL** (Class A shares) and **GOOG** (Class C shares).

These ticker symbols now refer to **Alphabet Inc.**, which is Google's holding company, since the fourth quarter of 2015. The company is also listed on the Frankfurt Stock Exchange under the ticker symbol **GGQ1**.

In [38]:
send_message("What is MGM Studio's stock symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

I am unable to find the stock symbol for MGM Studios.

In [39]:
send_message("What is MGM Studio's owner company stock symbol?")

send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The owner company of MGM Studios is Amazon, and its stock symbol is AMZN.


In [40]:
send_message("What is Facebook's stock ticker symbol?")

send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash
send_message: got a response


#### Natural language response

Facebook's stock ticker symbol is META.


In [41]:
send_message("""Compare Amazon's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response
Api.zero_error: model is now gemini-2.5-flash


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Amazon's recommendation trends from October 1, 2025, to November 29, 2025, show a consistently bullish outlook from analysts. In October, there were 23 "strong buy" and 52 "buy" recommendations, with 3 "hold" and no "sell" or "strong sell" recommendations. This positive sentiment continued into November, with 22 "strong buy" and 54 "buy" recommendations, and 2 "hold" recommendations. This indicates a sustained belief in Amazon's growth potential among analysts.

Sentiment analysis of news articles during the same period reveals a mixed but generally positive outlook for Amazon, with a strong emphasis on its AI and cloud computing (AWS) initiatives.

**Bullish Sentiments and Correlations:**

*   **AI and AWS Dominance:** Numerous articles highlight Amazon's strong position and significant investments in AI and AWS. The company's strategic partnerships (e.g., with OpenAI for a $38 billion cloud infrastructure deal), development of custom AI chips (Trainium, Inferentia), and expansion of AI services are consistently cited as major growth drivers. This correlates directly with the "strong buy" and "buy" recommendations, as analysts see these areas as key to future profitability.
*   **E-commerce and Advertising Growth:** Despite some mentions of slowing e-commerce growth in certain contexts, many articles emphasize Amazon's continued dominance in online retail (40% market share) and the rapid expansion of its digital advertising business (22% year-over-year growth). These segments are often seen as benefiting from AI integration, further reinforcing the positive outlook.
*   **Operational Efficiency and Cost Control:** Amazon's efforts to improve operational efficiency through robotics and AI, as well as strategic cost-cutting measures (e.g., layoffs in HR), are viewed positively by analysts as contributing to margin expansion and overall profitability.
*   **Strategic Partnerships and Investments:** Amazon's involvement in various partnerships, such as supporting AI startups through AWS programs and investing in companies like Rivian, are seen as diversifying its business and positioning it for long-term growth in emerging technologies.
*   **Attractive Valuation:** Several articles mention Amazon's stock trading at a "reasonable" or "attractive" valuation compared to its historical earnings multiples or peers, suggesting that even with its massive size, there's still upside potential.

**Bearish/Neutral Sentiments and Correlations:**

*   **Layoffs and Restructuring:** While some articles frame layoffs as a strategic move for AI-driven efficiency, others highlight the negative impact of job cuts on the workforce and potential financial challenges. This creates a neutral to slightly negative sentiment in some instances.
*   **Competition in Cloud and AI:** While Amazon's AWS is a leader, articles frequently mention intense competition from Microsoft Azure and Google Cloud, particularly in the AI space. The development of custom AI chips by competitors and partnerships like Anthropic's deal with Google Cloud are seen as potential threats to AWS's market dominance.
*   **Market Concentration and Valuation Concerns:** Some analyses, particularly those discussing the "Magnificent Seven" tech stocks, express concerns about overall market concentration and potential overvaluation in the tech sector, including Amazon. While not directly bearish on Amazon itself, these broader market concerns can create a cautious sentiment.
*   **Reduced Shipping Volumes with UPS:** UPS's strategic decision to reduce its reliance on Amazon for lower-margin deliveries is a recurring theme, indicating a shift in the relationship that could impact both companies. For Amazon, this is generally presented as a neutral development as it focuses on its own logistics network.
*   **AWS Outages:** A major global AWS outage due to a software bug in October generated negative sentiment, highlighting the risks associated with reliance on cloud infrastructure.

**Overall Patterns and Correlations:**

There's a strong correlation between Amazon's aggressive investments and advancements in AI and cloud computing, and the consistently bullish analyst recommendations. The news sentiment, while acknowledging competitive pressures and operational challenges, largely reinforces this positive outlook by highlighting the strategic importance and growth potential of these core business segments. The discussions around layoffs are often framed as a necessary step towards an AI-driven future, rather than a sign of fundamental weakness.

The market appears to be rewarding Amazon's long-term vision and execution in AI and cloud, even as it navigates a complex competitive landscape and makes strategic adjustments to its workforce and partnerships. The recurring theme is that Amazon is not just participating in the AI revolution but is actively shaping it, which underpins the sustained bullish sentiment.

In [42]:
send_message("""Compare Google's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.92it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: updating state
send_message: got a response
send_message: get function response
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp


#### Natural language response

I was able to retrieve information from the wiki tool regarding Google's bullish versus bearish predictions from October 1, 2025, until today, November 29, 2025.

Google's outlook as of October 1, 2025, presented a mix of strong growth potential and significant regulatory challenges.

**Bullish Sentiment:**

*   Driven by the $32 billion acquisition of Wiz (cybersecurity startup) in March 2025.
*   Commitment to AI, launching Bard in February 2025 and a £5 billion AI investment in the UK in September 2025.
*   Alphabet entered the $3 trillion market cap club in September 2025.
*   Stock experienced an 8% jump in September 2025 after avoiding severe penalties in an antitrust case.

**Bearish Sentiment:**

*   Persistent antitrust issues.
*   November 2024: The Justice Department argued for Google to sell Chrome.
*   April 2025: A judge ruled that Google maintained an illegal advertising monopoly.
*   August 2024: A judge found the company violated antitrust laws to preserve its dominance in online search.
*   Significant financial penalties from the European Union, including a €2.4 billion fine in September 2024 and another €2.95 billion fine in September 2025.
*   A UK competition watchdog found Google abusing its ad tech dominance in September 2024.

**Recommendation Trends:**

*   While explicit recommendation trends are not available, the market sentiment appears to be strongly positive regarding Google's advancements and investments in artificial intelligence, which significantly contributed to its market capitalization growth. However, there is a clear negative sentiment surrounding its ongoing legal battles and regulatory fines, which pose substantial financial and operational risks.

**Patterns and Correlations:**

*   Simultaneous occurrence of robust growth and innovation in artificial intelligence, which fuels market confidence, alongside continuous and significant regulatory and legal pressures concerning its market dominance in search and advertising.
*   The market's reaction to avoiding major asset divestitures, despite other antitrust findings, suggests that the integrity of Google's core platforms is a critical factor for investor confidence, balancing the tangible costs of fines with the potential for AI-driven expansion.


In [43]:
send_message("""How is the outlook for Apple based on trends and news sentiment from July 01 2025 until today? 
Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.""")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: get function response
send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
Api.zero_error: model is now gemini-2.5-flash
send_message: updating state
Api.refill_rpm 10
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating s

...

send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's an outlook for Apple and its peers based on trends and news sentiment from July 01, 2025, until today, November 29, 2025:

**Apple (AAPL) Outlook:**

*   **Analyst Recommendations:** Apple consistently receives a strong positive outlook from analysts. As of November 2025, recommendations are predominantly "Buy" (23) and "Strong Buy" (15), with a stable number of "Hold" (17) and very few "Sell" (2) or "Strong Sell" (0) ratings. This sentiment has remained largely consistent since August 2025.

*   **News Sentiment (July 01, 2025 - November 29, 2025):**
    *   **Positive Trends:** Apple has seen strong demand for its iPhone 17 series, with positive early sales and preorder trends. Its Services segment continues to grow robustly, contributing significantly to revenue. The company is making strategic investments in U.S. manufacturing and AI infrastructure, including the launch of its M5 chip and Apple Intelligence features. FDA approval for Apple Watch hypertension alerts and partnerships for rare-earth magnets also contribute to a positive outlook. Apple's financial performance and market capitalization remain strong, and it has successfully navigated some tariff policies.
    *   **Neutral/Mixed Trends:** There are ongoing concerns about Apple's pace of AI innovation compared to some competitors. The company is facing antitrust lawsuits and regulatory challenges in India and China, and class-action lawsuits regarding alleged misleading statements about Siri's AI capabilities. Warren Buffett has been gradually reducing his stake in Apple, though it remains a significant holding.
    *   **Negative Trends:** Perceived lack of significant innovation in new products and slower AI advancements are noted. There was also a report of reduced iPhone Air production due to weak demand.

**Apple's Peers by Sub-Industry Outlook:**

Apple's peers in the "Computer Hardware, Storage & Peripherals" and "Quantum Computing" sub-industries include: DELL, WDC, SNDK, PSTG, HPE, HPQ, NTAP, SMCI, IONQ, QUBT, and CMPO.

*   **Dell Technologies (DELL):**
    *   **Analyst Recommendations:** Consistently positive, with a high number of "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Strong AI server demand and revenue growth, raised guidance, and strategic partnerships are positive. However, a Morgan Stanley downgrade due to rising memory costs and margin pressures is a negative.

*   **Western Digital (WDC):**
    *   **Analyst Recommendations:** Consistently positive, with a high number of "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Recognized as a top S&P 500 performer with significant dividend increases, strong Q1 2026 earnings, and expansion of its System Integration and Test (SIT) Lab for AI storage.

*   **SanDisk (SNDK):**
    *   **Analyst Recommendations:** Consistently positive, with a high number of "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Significant stock price increase due to S&P 500 inclusion, strong performance in NAND flash memory and data center segments, and positive analyst upgrades. A prediction of falling memory chip prices in July was a negative.

*   **Pure Storage (PSTG):**
    *   **Analyst Recommendations:** Consistently positive, with a high number of "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Strong annual recurring revenue growth, key clients including Meta and Nvidia, and a subscription-based business model are positive.

*   **Hewlett Packard Enterprise (HPE):**
    *   **Analyst Recommendations:** Consistently positive, with a high number of "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Collaborating on quantum-safe SAN switch portfolios, strong telecommunications infrastructure, and successful acquisition of Juniper Networks are positive. Strategic restructuring costs compressing profit margins were a negative.

*   **HP Inc (HPQ):**
    *   **Analyst Recommendations:** Generally neutral to positive, with a high number of "Hold" ratings.
    *   **News Sentiment:** Leading production in Southeast Asia, producing sustainable devices, and launching innovative gaming products are positive. Workforce reduction and missing revenue forecasts are negative.

*   **NetApp (NTAP):**
    *   **Analyst Recommendations:** Generally neutral to positive, with a high number of "Hold" ratings.
    *   **News Sentiment:** Aligned with quantum-safe security and AI-powered management for enterprise storage networks, and mentioned in sustainability discussions are positive.

*   **Super Micro Computer (SMCI):**
    *   **Analyst Recommendations:** Mixed, with a good number of "Buy" and "Hold" ratings, and some "Sell" ratings.
    *   **News Sentiment:** Launched integrated AI factory systems with Nvidia's Blackwell technology and strong AI order backlog are positive. However, missed revenue and earnings expectations, declining margins, negative cash flow, and being a heavily shorted stock are significant negatives.

*   **IonQ (IONQ) & Quantum Computing Inc (QUBT) (Quantum Computing):**
    *   **Analyst Recommendations:** Both show consistently positive "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Both companies are highly speculative. They have seen significant stock surges due to technological breakthroughs, government interest, and strategic partnerships. However, they face substantial risks due to high valuations, unproven commercial viability, significant operating losses, and minimal sales. IonQ has seen significant pullbacks due to these concerns.

*   **CompoSecure (CMPO):**
    *   **Analyst Recommendations:** Consistently positive, with a high number of "Buy" and "Strong Buy" ratings.
    *   **News Sentiment:** Neutral, with news focusing on a planned business combination and ongoing management fee generation.

**Comparison of Apple to its Peers:**

*   **AI Strategy:** Apple's AI strategy is more focused on integrating AI into its consumer ecosystem (Apple Intelligence, Siri upgrades, M5 chip). Many of its hardware peers (DELL, WDC, SNDK, PSTG, HPE, SMCI) are foundational to the broader AI boom, providing the critical infrastructure like AI servers, storage, and chips.
*   **Market Position & Stability:** Apple, along with established hardware peers like Dell and Western Digital, maintains a strong and stable market position with consistent positive analyst sentiment.
*   **Valuation & Risk:** While Apple faces some concerns about its valuation and AI pace, the pure-play quantum computing peers (IONQ, QUBT) carry significantly higher risk due to their nascent technology, extreme valuations relative to revenue, and unproven commercial viability.
*   **Legal & Regulatory:** Apple is more frequently mentioned in news related to antitrust lawsuits and class-action lawsuits concerning its AI capabilities, indicating a higher level of regulatory and legal scrutiny compared to most of its hardware peers.
*   **Growth Drivers:** For Apple, growth is driven by product cycles (iPhone 17), services expansion, and strategic AI integration. For many hardware peers, growth is directly tied to the massive demand for AI infrastructure.

**Overall Conclusion:**

Apple's outlook remains generally positive, supported by strong product demand, a growing services segment, and strategic investments in AI. However, it faces challenges in accelerating its AI innovation and navigating regulatory pressures.

Many of Apple's established hardware peers are also experiencing a very strong positive outlook, largely fueled by the insatiable demand for AI infrastructure. Companies like Dell, Western Digital, and SanDisk are showing impressive growth and positive sentiment in this area. The quantum computing peers (IonQ, Quantum Computing Inc.) present high-risk, high-reward opportunities, with significant technological advancements but also substantial financial and market uncertainties.

In essence, while Apple is a strong and stable company with a positive outlook, some of its peers in the AI infrastructure space are currently demonstrating more explosive growth and positive news sentiment, driven by the foundational needs of the AI revolution. Apple's ability to effectively integrate and monetize AI within its ecosystem will be crucial for its continued outperformance relative to these rapidly evolving peers.

In [44]:
send_message("""What does the recent news say about Apple and the impact of tariffs? From 2025-09-01 up to today. 
Also locate candlestick data for the same dates. 
Discuss in detail any correlations in patterns between the candlestick and news data. Ignore duplicate news entry.""")

send_message: get function response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.refill_rpm 10
send_message: got a response


#### Natural language response

Here's a detailed look at the recent news regarding Apple and the impact of tariffs, along with candlestick data for the same period (September 1, 2025, to November 29, 2025), and an analysis of their correlations.

### Recent News about Apple and the Impact of Tariffs (September 1, 2025 - November 29, 2025)

Several news articles during this period discussed tariffs and their potential or actual impact on Apple:

**Negative Impacts/Concerns:**

*   **October 26, 2025:** News highlighted potential significant challenges for Apple due to US-China trade tensions, including the risk of 100% tariffs and export restrictions on rare earth elements, which could disrupt iPhone component supply chains.
*   **October 13, 2025:** Reports indicated that Apple's stock performance was being impacted by escalating trade barriers between the United States and China.
*   **November 18, 2025:** Apple reported substantial tariff-related cost increases, amounting to $1.1 billion, with expectations of this figure rising to $1.4 billion.
*   **October 10, 2025:** President Trump's renewed tariff threats against China triggered a significant market selloff, causing Apple's stock price to decline as part of a broader market reaction.
*   **September 4, 2025:** Early reports mentioned Apple facing potential tariff threats.

**Mitigation Strategies/Positive Adaptations:**

*   **October 28, 2025:** News suggested that Apple successfully navigated arduous tariffs through strategic U.S. investments, relocating iPhone production, and securing exemptions from Chinese and Indian tariffs.
*   **October 31, 2025:** Apple's proactive approach to supply chain risks was highlighted, with an investment of $500 million in MP Materials for rare earth magnet recycling facility development, addressing concerns related to China's export controls.
*   **October 6, 2025:** Apple was noted for striking side deals to mitigate tariff impacts, showcasing its strategic adaptability.
*   **Various dates (September 4, 6, 12, October 16, November 2, 17, 19, 26, 29):** Multiple articles mentioned Apple's strategic partnerships and investments with MP Materials to secure a domestic supply of rare earth materials, aiming to reduce dependency and mitigate risks from Chinese export controls.
*   **September 18 & 20, 2025:** Apple announced a $500 billion investment in AI over four years, a long-term technological commitment that could enhance its competitive position amidst global trade tensions.

### Candlestick Data for Apple (AAPL) (September 1, 2025 - November 29, 2025)

The candlestick data for Apple (AAPL) during this period shows an overall upward trend, despite some daily fluctuations:

*   **Early September (Sep 2-5):** The stock generally trended upwards, closing around $239.69 on Sep 5 from an opening of $229.25 on Sep 2.
*   **Mid-September (Sep 15-19):** A continued upward movement was observed, with the stock closing around $245.5 on Sep 19 from a close of $236.7 on Sep 15.
*   **Late September/Early October (Sep 29 - Oct 3):** The stock saw a notable increase, closing at $258.02 on Oct 3 from $254.43 on Sep 29.
*   **October 10, 2025:** The stock experienced a significant drop, opening at $254.94 and closing at $245.27, with a low of $244.0.
*   **Mid-October (Oct 13-17):** The stock remained in a lower range after the Oct 10 drop, closing at $247.66 on Oct 13 and $252.29 on Oct 17.
*   **Late October (Oct 27-31):** The stock showed a strong rebound and upward trend, closing at $268.81 on Oct 27 and $270.37 on Oct 31.
*   **Mid-November (Nov 10-14):** The stock generally moved upwards, closing at $269.43 on Nov 10 and $272.41 on Nov 14.
*   **Late November (Nov 17-28):** The stock continued its upward trajectory, closing at $267.44 on Nov 17 and reaching $278.85 on Nov 28.

### Correlations in Patterns between Candlestick and News Data

There are observable correlations between the tariff-related news and Apple's stock performance:

*   **Negative News and Price Dips:**
    *   The news on **October 10, 2025**, about "Trump Shocks Markets" with renewed tariff threats, directly correlates with a significant single-day drop in Apple's stock price, as seen in the candlestick data for that day (open $254.94, close $245.27).
    *   Similarly, the report on **November 18, 2025**, detailing Apple's $1.1 billion in tariff-related cost increases, aligns with a downward movement in the stock price on that day (open $269.99, close $267.44, with a notable low of $265.32).
    *   The general negative sentiment around escalating trade barriers on **October 13, 2025**, also corresponds with the stock remaining in a lower range after the sharp drop on October 10.

*   **Positive/Mitigation News and Price Resilience/Uptrends:**
    *   The news on **October 28, 2025**, highlighting Apple's success in "Powering Through Trump’s Tariff Policies" by adapting strategies, coincides with a positive day for the stock (open $269.275, close $269.7), suggesting investor confidence in Apple's ability to manage tariff challenges.
    *   Apple's strategic investments in domestic rare earth material supply (mentioned across various dates, particularly around **late October and November** with MP Materials) likely contributed to the overall upward trend and resilience of the stock during this period, as these actions demonstrate a proactive approach to mitigating future supply chain risks related to trade tensions.
    *   The broader positive news around iPhone 17 launches and AI investments (e.g., mid-September and late September/early October) also contributed to the overall bullish sentiment, helping to offset some of the tariff-related concerns and driving the stock to higher levels by the end of November.

In conclusion, while direct negative news about tariffs or their financial impact on Apple led to immediate downward pressure on the stock, Apple's proactive and strategic responses to these challenges, coupled with positive product and AI developments, appear to have instilled confidence in investors, leading to an overall upward trend in its stock price during the September to November 2025 period. The market seems to reward Apple's adaptability and long-term strategic planning in navigating a complex global trade environment.

# StockChat: Agents Edition

In [5]:
# Setup working directory on Kaggle.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    if not os.path.isdir("sc2/"):
        !git init -b main
        !git remote add origin https://github.com/lol-dungeonmaster/kaggle-agents-2025.git
        !git config core.sparseCheckout true
        !echo "sc2/" >> .git/info/sparse-checkout
        !git pull origin main
        for api_key in ["GOOGLE_API_KEY","POLYGON_API_KEY","FINNHUB_API_KEY"]:
            env_key = UserSecretsClient().get_secret(api_key)
            !echo "$api_key=$env_key" >> sc2/.env # from .venv on local runs
            os.environ[api_key] = UserSecretsClient().get_secret(api_key) # from .venv on local runs

Initialized empty Git repository in /kaggle/working/.git/
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 517 (delta 137), reused 171 (delta 99), pack-reused 303 (from 1)
Receiving objects: 100% (517/517), 472.93 KiB | 1.37 MiB/s, done.
Resolving deltas: 100% (329/329), done.
From https://github.com/lol-dungeonmaster/kaggle-agents-2025
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [5]:
# Define async helper functions.
from sc2.agent import app
from sc2.src import log

if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    log_info = print
    log_warn = print
    log_err = print
else:
    log_info = log.info
    log_warn = log.warning
    log_err = log.error

async def on_event(e: Event, q: str):
    try:
        response = e.content.parts[0].text
        if response and response != "None":
            log_info(f"USER  > {q}")
            log_info(f"MODEL > {response}\n")
    except Exception as err:
        log_err(f"on_event.exception: {str(err)}")

async def run_queries(app: App, sessions: SessionService, queries: list[str],
                      session_id: str = "default", user_id: str = "default"):
    runner = Runner(app=app, session_service=sessions)
    try:
        session = await sessions.create_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    except:
        session = await sessions.get_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    finally:
        log_info(f"### Agent session: (uid={user_id}) {session_id}\n")
        for query in queries:
            await try_run(runner, session, user_id, query)

async def try_run(runner: Runner, session: Session, user_id: str, query: str):
    try:
        q = types.Content(role="user", parts=[types.Part(text=query)])
        async for response in runner.run_async(
            user_id=user_id, session_id=session.id, new_message=q
        ): await on_event(response, query)
    except Exception as e:
        q_id = " ".join(query.split()[:4])
        if type(e) in [KeyError, IndexError]:
            log_warn(f"try_run.run_async (q={q_id}): retrying, generated {type(e).__name__}\n")
            time.sleep(5.0)
            await try_run(runner, session, user_id, query)
        else:
            log_warn(f"try_run.run_async (q={q_id}): {type(e).__name__} - {str(e)}\n")

async def check_compaction(sessions: SessionService, session_id: str = "default", 
                           user_id: str = "default", show_llm: bool = False):
    n = 0
    for e in (await sessions.get_session(
        app_name=app.name,
        user_id=user_id,
        session_id=session_id,
    )).events:
        if e.actions and (llm_out := e.actions.compaction):
            n += 1
            if show_llm:
                log_info(f"check_compaction.show_llm: {llm_out.compacted_content.parts[0].text}\n")
    log_info(f"check_compaction: found ({n}) compaction event\n")

WARNING  [root] KeyError: authentication token for LMNR_PROJECT_API_KEY is undefined
INFO     [root] Skipping Laminar.initialize()
INFO     [root] sc2.__init__: the api-helper is ready
Generate document embedding: 100%|##########| 1/1 [00:04<00:00,  4.29s/it]
INFO     [root] sc2.__init__: RestGroundingTool is ready
INFO     [root] sc2.__init__: SearchGroundingTool is ready
INFO     [root] sc2.__init__: WikiGroundingTool is ready
INFO     [root] sc2.__init__: MemoryService is ready
WARNING  [root] [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


In [6]:
# Create a session service and run some test queries.
s_svc = InMemorySessionService()

await run_queries(
    app=app, sessions=s_svc,
    queries=[
        "What tools do you know how to use?",
        "Ask `sc2_fnplan` what functions fncall_pipeline knows.",
        "What is a short trade?",
        "What is gambler's ruin?",
        "My local advisor is SC at JPMorgan Chase, 212-736-2001",
        "I live in Brooklyn, New York."])

INFO     [root] ### Agent session: (uid=default) default

INFO     [root] USER  > What tools do you know how to use?
INFO     [root] MODEL > I can use the following tools:

*   **sc2_memory**: To write long-term memories.
*   **sc2_prefs**: To analyze and store profile data.
*   **fncall_pipeline**: To call functions defined in `sc2_fnplan`.
*   **sc2_fnplan**: To plan function calls.
*   **sc2_terms**: To find and define financial terms.
*   **sc2_summary**: To create concise summaries.

INFO     [root] USER  > Ask `sc2_fnplan` what functions fncall_pipeline knows.
INFO     [root] MODEL > `fncall_pipeline` knows the following functions:

*   **get_symbol_1**: Search for the stock ticker symbol of a given company, security, isin or cusip.
*   **get_symbols_1**: List all supported symbols and tickers, filtered by exchange code.
*   **get_name_1**: Search for the name associated with a stock ticker or symbol's company, security, isin or cusip.
*   **get_symbol_quote_1**: Search for the c

In [ ]:
# Use long-term memory from a new session.
s_svc2 = InMemorySessionService()

await run_queries(
    app=app, sessions=s_svc2,
    queries=[
        "Check memory for where I live.",
        "Check memory for my local advisor SCs phone number."])

INFO     [root] ### Agent session: (uid=default) default

INFO     [root] USER  > Check memory for where I live.
INFO     [root] MODEL > You live in Brooklyn, New York.

INFO     [root] USER  > Check memory for my local advisor SCs phone number?
INFO     [root] MODEL > SC's phone number is 212-736-2001. SC works at JPMorgan Chase.



INFO     [root] Api.refill_rpm 10


In [13]:
# Display context compaction output.
await check_compaction(s_svc, show_llm=True)

INFO     [root] check_compaction.show_llm: The conversation began with the user inquiring about the AI's capabilities. The AI listed its available tools: `sc2_memory`, `sc2_prefs`, `fncall_pipeline`, `sc2_fnplan`, `sc2_terms`, and `sc2_summary`.

The user then asked `sc2_fnplan` to list the functions known by `fncall_pipeline`. The AI responded with a comprehensive list of functions, primarily related to financial data, market status, company information (peers, financials, overview), stock quotes, candlestick data, news, recommendation trends, and grounding capabilities (Wikipedia and internet search).

Finally, the user asked for a definition of a "short trade." The AI provided a detailed explanation, describing it as an investment strategy where borrowed shares are sold with the expectation of buying them back at a lower price to profit from a price decline. It also highlighted the higher risks associated with this strategy, including unlimited potential losses.

**Key Information &

## Run evaluation

In [5]:
!adk eval sc2 sc2/eval/test_cases.json --config_file_path=sc2/eval/config.json --print_detailed_results

/home/sysop/Documents/kaggle-agents-2025/.venv/lib/python3.12/site-packages/google/adk/evaluation/metric_evaluator_registry.py:90: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
/home/sysop/Documents/kaggle-agents-2025/.venv/lib/python3.12/site-packages/google/adk/evaluation/local_eval_service.py:80: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
Using evaluation criteria: criteria={'rubric_based_final_response_quality_v1': BaseCriterion(threshold=0.8, judge_model_options={'judge_model': 'gemini-2.5-flash', 'num_samples': 1}, rubrics=[{'rubric_id': 'conciseness', 'rubric_